**Classification of XRays using Machine Learning**
---
My name is Teddy John Kamau Mbiri, a final year Computer Science student at University of West London. 
This is my final year project, the use of Machine Learning to classify X-rays, whether a patient has pneumonia, covid pneumonia or neither.


**Data Exploration**
---
The section below will explore the data and the features of the dataset.
I will be using data from multiple sources and previous work to create a model that can classify X-rays.

**Data Source**
1.	https://drive.google.com/drive/folders/128hxYxQ8kVEkSkVGikueiqBxWTmDIW2h
2.	https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia
3.	https://www.kaggle.com/nikhilpandey360/chest-xray-masks-and-labels
4.  https://data.mendeley.com/datasets/jctsfj2sfn/1
5. https://www.rsna.org/education/ai-resources-and-training/ai-image-challenge/rsna-pneumonia-detection-challenge-2018



In [1]:
#split the data into train and test and validation
import splitfolders
#Library required for the splitfolders function
splitfolders.ratio("C:/Users/tedjo/Downloads/XRAY", output="C:/Users/tedjo/Downloads/XRAY/SplitData", seed=1337, ratio=(.8, .1, .1))
#SplitData is the folder where the data will be stored

Copying files: 45001 files [00:35, 1261.44 files/s]


In [2]:
#After the data is split and stored in the SplitData folder, the next step is to explore the data.
#import required libraries and there functions within my code
import pandas as pd #dataframe
import numpy as np #numerical python
import matplotlib.pyplot as plt #plotting 
import seaborn as sns #plotting
import os #operating system
from tensorflow.keras.preprocessing import image #image processing
from PIL import Image #image processing


In [12]:
pip install tensorflow-gpu==2.9.1


  Using cached libclang-14.0.1-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Using cached tensorboard-2.9.0-py3-none-any.whl (5.8 MB)
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.15.0
    Uninstalling absl-py-0.15.0:
      Successfully uninstalled absl-py-0.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.0
    Uninstalling tensorboard-2.4.0:
      Successfully uninstalled tensorboard-2.4.0
Note: you may need

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.3.0 requires gast==0.3.3, but you have gast 0.4.0 which is incompatible.
tensorflow 2.3.0 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
tensorflow 2.3.0 requires scipy==1.4.1, but you have scipy 1.7.3 which is incompatible.
tensorflow 2.3.0 requires tensorflow-estimator<2.4.0,>=2.3.0, but you have tensorflow-estimator 2.9.0 which is incompatible.


In [14]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices())

2.3.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [ ]:
#Converting the images to a matrix
def imgtonp(path, list_of_filename,size = (64,64)):
    #Iterating through each file
    for fn in list_of_filename:
        fp = path + fn
        current_image = image.load_img(fp, target_size=size)
    #Converting the image to a matrix
        image_to_matrix = image.img_to_array(current_image)#Convert image to matrix
    #Convert into a vector
        image_to_matrix= [image_to_matrix.ravel()]
        try:
            #concatenate through the various images
            img_mat = np.concatenate((img_mat, image_to_matrix))
        except UnboundLocalError:
            img_mat = image_to_matrix
    return img_mat

In [ ]:
def average_img(img_mat,title):
    #Average the image
    img_mat = np.mean(img_mat,axis=0)
    #Reshape the image
    img_mat = img_mat.reshape(64,64,3)
    #Plot the image
    plt.imshow(img_mat,vmin=0,vmax=255,cmap='gray')
    plt.title(f'Average {title}')
    plt.show()
    return img_mat

In [ ]:
#After splitting the images into separate folders plotting a histogram of the images in the folders will help identify imbalance in the classes
#First is creating a dictionary to store the number of images in each folder
#Import the data from the train folder
train_path = "C:/Users/tedjo/Downloads/XRAY/SplitData/train"
Xray_classes = {'covid19_img': len(os.listdir(f'{train_path}/COVID')),
                'normal_img': len(os.listdir(f'{train_path}/NORMAL')),
                'pneumo_img': len(os.listdir(f'{train_path}/PNEUMONIA'))}

#Plotting the histogram of the images in the folders
plt.bar(Xray_classes.keys(), Xray_classes.values(),color=['red','green','blue'])
plt.title('Count of images in each train class')
plt.xlabel('Class')
plt.ylabel('Count')
#The use of a for loop to iterate through the folders and count the number of images in each folder
for key, value in Xray_classes.items():
    plt.text(key, value, value)

In [ ]:
#Import the data from the train folder
train_path = "C:/Users/tedjo/Downloads/XRAY/SplitData/train"

#Create a list of all the images in the train folder
covid19_img = [fn for fn in os.listdir(f'{train_path}/COVID') if fn.endswith('.jpg')]
normal_img = [fn for fn in os.listdir(f'{train_path}/NORMAL') if fn.endswith('.jpg')]
pneumo_img = [fn for fn in os.listdir(f'{train_path}/PNEUMONIA') if fn.endswith('.jpg')]

#Select 3 random images from each class to display from the train folder
covid_img = np.random.choice(covid19_img, 3, replace = False)
normal_img = np.random.choice(normal_img, 3, replace = False)
pneumo_img = np.random.choice(pneumo_img, 3, replace = False)

#PLot a matrix of the images
fig = plt.figure(figsize=(10,10))
for i in range(9):
    if i < 3: 
        fp = f'{train_path}/COVID/{covid_img[i]}'
        label = 'COVID19'
    elif i>=3 and i<6:
        fp = f'{train_path}/NORMAL/{normal_img[i-3]}'
        label = 'NORMAL'
    else:
        fp = f'{train_path}/PNEUMONIA/{pneumo_img[i-6]}'
        label = 'PNEUMONIA'
    ax = fig.add_subplot(3,3,i+1)
    
    #Load the image
    fn = image.load_img(fp, target_size=(224,224), color_mode='grayscale')
    plt.imshow(fn,cmap='Greys_r')
    plt.title(label)
    plt.axis('off')
plt.show()




In [ ]:
#Convert train images to a matrix
train_covid19_images = imgtonp(f'{train_path}/COVID/', covid_img)
train_normal_images = imgtonp(f'{train_path}/NORMAL/', normal_img)
train_pneumo_images = imgtonp(f'{train_path}/PNEUMONIA/', pneumo_img)

In [ ]:
covid_average = average_img(train_covid19_images, 'COVID19')
normal_average = average_img(train_normal_images, 'NORMAL')
pneumo_average = average_img(train_pneumo_images, 'PNEUMONIA')

In [ ]:
test_path = "C:/Users/tedjo/Downloads/XRAY/SplitData/test"
Xray_classes = {'covid19_img': len(os.listdir(f'{test_path}/COVID')),
                'normal_img': len(os.listdir(f'{test_path}/NORMAL')),
                'pneumo_img': len(os.listdir(f'{test_path}/PNEUMONIA'))}

#Plotting the histogram of the images in the folders
plt.bar(Xray_classes.keys(), Xray_classes.values(),color=['red','green','blue'])
plt.title('Count of images in each test class')
plt.xlabel('Class')
plt.ylabel('Count')
#The use of a for loop to iterate through the folders and count the number of images in each folder
for key, value in Xray_classes.items():
    plt.text(key, value, value)


In [ ]:
# sourcery skip: remove-redundant-if
test_path = "C:/Users/tedjo/Downloads/XRAY/SplitData/test"

#Create a list of all the images in the test folder
covid19_img = [fn for fn in os.listdir(f'{test_path}/COVID') if fn.endswith('.jpg')]
normal_img = [fn for fn in os.listdir(f'{test_path}/NORMAL') if fn.endswith('.jpg')]
pneumo_img = [fn for fn in os.listdir(f'{test_path}/PNEUMONIA') if fn.endswith('.jpg')]

#Select 3 random images from each class to display from the test folder
covid_img = np.random.choice(covid19_img, 3, replace = False)
normal_img = np.random.choice(normal_img, 3, replace = False)
pneumo_img = np.random.choice(pneumo_img, 3, replace = False)

#PLot a matrix of the images
fig = plt.figure(figsize=(10,10))
for i in range(9):
    if i <3:
        fp = f'{test_path}/COVID/{covid_img[i]}'
        label = 'COVID19'
    elif i>=3 and i<6:
        fp = f'{test_path}/NORMAL/{normal_img[i-3]}'
        label = 'NORMAL'
    else:
        fp = f'{test_path}/PNEUMONIA/{pneumo_img[i-6]}'
        label = 'PNEUMONIA'
    ax = fig.add_subplot(3,3,i+1)

    #load the image
    fn = image.load_img(fp, target_size=(224,224), color_mode='grayscale')
    plt.imshow(fn,cmap='Greys_r')
    plt.title(label)
    plt.axis('off')
plt.show()

        

In [ ]:
test_covid_images = imgtonp(f'{test_path}/COVID/', covid_img)
test_normal_images = imgtonp(f'{test_path}/NORMAL/', normal_img)
test_pneumo_images = imgtonp(f'{test_path}/PNEUMONIA/', pneumo_img)

In [ ]:
val_path = "C:/Users/tedjo/Downloads/XRAY/SplitData/val"
Xray_classes = {'covid19_img': len(os.listdir(f'{val_path}/COVID')),
                'normal_img': len(os.listdir(f'{val_path}/NORMAL')),
                'pneumo_img': len(os.listdir(f'{val_path}/PNEUMONIA'))}

plt.bar(Xray_classes.keys(), Xray_classes.values(),color=['red','green','blue'])
plt.title('Count of images in each val class')
plt.xlabel('Class')
plt.ylabel('Count')
#The use of a for loop to iterate through the folders and count the number of images in each folder
for key, value in Xray_classes.items():
    plt.text(key, value, value)

In [ ]:
# sourcery skip: remove-redundant-if
val_path = "C:/Users/tedjo/Downloads/XRAY/SplitData/val"
covid19_img = [fn for fn in os.listdir(f'{val_path}/COVID') if fn.endswith('.jpg')]
normal_img = [fn for fn in os.listdir(f'{val_path}/NORMAL') if fn.endswith('.jpg')]
pneumo_img = [fn for fn in os.listdir(f'{val_path}/PNEUMONIA') if fn.endswith('.jpg')]

covid_img = np.random.choice(covid19_img, 3, replace = False)
normal_img = np.random.choice(normal_img, 3, replace = False)
pneumo_img = np.random.choice(pneumo_img, 3, replace = False)

fig = plt.figure(figsize=(10,10))
for i in range(9):
    if i < 3:
        fp = f'{val_path}/COVID/{covid_img[i]}'
        label = 'COVID19'
    elif i>=3 and i<6:
        fp = f'{val_path}/NORMAL/{normal_img[i-3]}'
        label = 'NORMAL'
    else:
        fp = f'{val_path}/PNEUMONIA/{pneumo_img[i-6]}'
        label = 'PNEUMONIA'
    ax  = fig.add_subplot(3,3,i+1)
    fn = image.load_img(fp, target_size=(224,224), color_mode='grayscale')
    plt.imshow(fn,cmap='Greys_r')
    plt.title(label)
    plt.axis('off')
plt.show()



In [ ]:
val_covid_images = imgtonp(f'{val_path}/COVID/', covid_img)
val_normal_images = imgtonp(f'{val_path}/NORMAL/', normal_img)
val_pneumo_images = imgtonp(f'{val_path}/PNEUMONIA/', pneumo_img)

In [ ]:
from sklearn.decomposition import PCA
from math import ceil

def eigenimages(img_mat,title,n_comp=0.7,size=(64,64,3)):
    pca = PCA(n_components=n_comp,whiten=True)
    pca.fit(img_mat)
    print('Number of PC: ',pca.n_components_)
    return pca
def plot_pca(pca,size=(64,64,3)):
    n = pca.n_components_
    fig = plt.figure(figsize=(10,10))
    r = int(n**0.5)
    c = ceil(n/r)
    for i in range(n):
        ax = fig.add_subplot(r,c,i+1,xticks=[],yticks=[])
        ax.imshow(pca.components_[i].reshape(size),cmap='Greys_r')
    plt.axis('off')
    plt.show()

In [ ]:
plot_pca(eigenimages(train_covid19_images,'COVID'))
plot_pca(eigenimages(train_normal_images,'NORMAL'))
plot_pca(eigenimages(train_pneumo_images,'PNEUMONIA'))
plot_pca(eigenimages(test_covid_images,'COVID'))